In [67]:
import hopsworks
import pandas as pd

project = hopsworks.login()
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://snurran.hops.works/p/16503
Connected. Call `.close()` to terminate connection gracefully.


In [70]:
df = pd.read_csv("../data/Mensa_occupancy_history.csv", sep=";")
df['city']="Heilbronn"
df['building']="Mensa Gebäude U8"

In [71]:
df = df.rename(columns={'Kapazität': 'capacity', 'Timestamp' : 'ts', 'Auslastung' : 'y'})
df['ds'] = pd.to_datetime(df['ts'], format='%d.%m.%y %H:%M')
df

,ts,y,capacity,city,building,ds
0,25.06.24 12:01,198.0,NaN,Heilbronn,Mensa Gebäude U8,2024-06-25 12:01:00
1,25.06.24 12:02,214.0,NaN,Heilbronn,Mensa Gebäude U8,2024-06-25 12:02:00
2,25.06.24 12:03,249.0,NaN,Heilbronn,Mensa Gebäude U8,2024-06-25 12:03:00
3,25.06.24 12:04,279.0,NaN,Heilbronn,Mensa Gebäude U8,2024-06-25 12:04:00
4,25.06.24 12:05,287.0,NaN,Heilbronn,Mensa Gebäude U8,2024-06-25 12:05:00
...,...,...,...,...,...,...
50019,13.08.24 13:18,54.0,500.0,Heilbronn,Mensa Gebäude U8,2024-08-13 13:18:00
50020,13.08.24 13:19,54.0,500.0,Heilbronn,Mensa Gebäude U8,2024-08-13 13:19:00
50021,13.08.24 13:20,53.0,500.0,Heilbronn,Mensa Gebäude U8,2024-08-13 13:20:00
50022,13.08.24 13:21,47.0,500.0,Heilbronn,Mensa Gebäude U8,2024-08-13 13:21:00


In [72]:
fg = fs.get_or_create_feature_group(name="mensa", 
                                    version=1, 
                                    description="Time-series capacity load measurements for a Mensa restaurant",
                                    primary_key=["city", "building", "ts"], 
                                    event_time="ds",
                                    online_enabled=True
                                   )
fg.insert(df)

Feature Group created successfully, explore it at 
https://snurran.hops.works/p/16503/fs/16451/fg/17442


Uploading Dataframe: 0.00% |          | Rows 0/50024 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: mensa_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://snurran.hops.works/p/16503/jobs/named/mensa_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7faebafaf160>, None)

In [73]:
df2= pd.read_csv("../data/Temprature_Heilbronn_history.csv", sep=";")
df2['city']="Heilbronn"
df2['ts'] = pd.to_datetime(df2['Timestamp'], format='%Y-%m-%d %H:%M:%S')
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Timestamp           50000 non-null  object        
 1   outdoortemperature  50000 non-null  float64       
 2   city                50000 non-null  object        
 3   ts                  50000 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 1.5+ MB


In [74]:
df2

,Timestamp,outdoortemperature,city,ts
0,2024-07-09 20:47:31,31.400000,Heilbronn,2024-07-09 20:47:31
1,2024-07-09 20:48:31,31.400000,Heilbronn,2024-07-09 20:48:31
2,2024-07-09 20:49:31,31.299999,Heilbronn,2024-07-09 20:49:31
3,2024-07-09 20:50:31,31.299999,Heilbronn,2024-07-09 20:50:31
4,2024-07-09 20:51:31,31.299999,Heilbronn,2024-07-09 20:51:31
...,...,...,...,...
49995,2024-08-13 13:38:26,38.200001,Heilbronn,2024-08-13 13:38:26
49996,2024-08-13 13:39:26,38.200001,Heilbronn,2024-08-13 13:39:26
49997,2024-08-13 13:40:26,38.200001,Heilbronn,2024-08-13 13:40:26
49998,2024-08-13 13:41:26,38.400002,Heilbronn,2024-08-13 13:41:26


In [75]:
fg = fs.get_or_create_feature_group(name="heilbronn_temperature", 
                                    version=1, 
                                    description="Temperature readings at Heibronn",
                                    primary_key=["city", "timestamp"], 
                                    event_time="ts",
                                    online_enabled=True
                                   )
fg.insert(df2)

2024-08-18 19:45:45,651 WARNING: FeatureGroupWarning: The ingested dataframe contains upper case letters in feature names: `['Timestamp']`. Feature names are sanitized to lower case in the feature store.



Uploading Dataframe: 0.00% |          | Rows 0/50000 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: heilbronn_temperature_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://snurran.hops.works/p/16503/jobs/named/heilbronn_temperature_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7faeb20296f0>, None)

In [76]:
df3= pd.read_csv("../data/waste_bin_filling_level_history.csv", sep=";")
df3['city']="Heilbronn"
df3['ts'] = pd.to_datetime(df3['Timestamp'], format='%Y-%m-%d %H:%M:%S')
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1815 entries, 0 to 1814
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Timestamp  1815 non-null   object        
 1   fillLevel  1815 non-null   int64         
 2   city       1815 non-null   object        
 3   ts         1815 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 56.8+ KB


In [77]:
fg = fs.get_or_create_feature_group(name="waste_bin_filling_level", 
                                    version=1, 
                                    description="Waste bin filling levelat Heibronn",
                                    primary_key=["city", "timestamp"], 
                                    event_time="ts",
                                   )
fg.insert(df3)

2024-08-18 19:45:48,571 WARNING: FeatureGroupWarning: The ingested dataframe contains upper case letters in feature names: `['Timestamp', 'fillLevel']`. Feature names are sanitized to lower case in the feature store.



Uploading Dataframe: 0.00% |          | Rows 0/1815 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: waste_bin_filling_level_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://snurran.hops.works/p/16503/jobs/named/waste_bin_filling_level_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7faf7ceb7490>, None)